<a href="https://colab.research.google.com/github/karishita/ML_Project2_ForestCover/blob/main/CovType_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv('/content/covtype_clean.csv')

In [ ]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,soil_type,wilderness_area
0,2596,51,3,258,0,510,221,232,148,6279,5,29,1
1,2590,56,2,212,-6,390,220,235,151,6225,5,29,1
2,2804,139,9,268,65,3180,234,238,135,6121,2,12,1
3,2785,155,18,242,118,3090,238,238,122,6211,2,30,1
4,2595,45,2,153,-1,391,220,234,150,6172,5,29,1


In [ ]:
df.shape

(581012, 13)

In [ ]:
X=df.drop(columns=['Cover_Type'])
y=df['Cover_Type']

In [ ]:
numeric_cols=['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points']

In [ ]:
#Splitting into 70 % train data 30% test
from sklearn.model_selection import train_test_split
X_train70_raw, X_test30_raw, y_train70, y_test30 = train_test_split(
    X, y, test_size=0.30, random_state=42
)



In [ ]:
#Validation Set 20% of training
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(
    X_train70_raw, y_train70, test_size=0.20, random_state=42
)


In [ ]:
#Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_raw[numeric_cols] = scaler.fit_transform(X_train_raw[numeric_cols])
X_valid_raw[numeric_cols] = scaler.transform(X_valid_raw[numeric_cols])
X_test30_raw[numeric_cols] = scaler.transform(X_test30_raw[numeric_cols])
X_train = X_train_raw
X_valid = X_valid_raw
X_test  = X_test30_raw

In [ ]:
!pip install optuna


In [ ]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
def objective(trial):

    params = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "max_depth": trial.suggest_int("max_depth", 2, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", [None, "sqrt", "log2"]),
    }

    # Force balanced class weights
    model = DecisionTreeClassifier(
        **params,
        class_weight="balanced",
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_valid)

    return accuracy_score(y_valid, preds)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("\nBest Parameters:", study.best_params)
print("Best Validation Score:", study.best_value)

# -----------------------------
#  Training Best Model on the entire 56% train split
# -----------------------------
best_params = study.best_params
best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# -----------------------------
#  Final Evaluation on 30% Test Set
# -----------------------------
y_pred = best_model.predict(X_test)

print("\nTest Accuracy:", accuracy_score(y_test30, y_pred))
print("\nClassification Report:\n", classification_report(y_test30, y_pred))
print("Confusion Matrix\n",confusion_matrix(y_test30, y_pred))

[I 2025-11-17 12:11:17,715] A new study created in memory with name: no-name-51bcfeba-2333-4386-83d5-e2b838798d49
[I 2025-11-17 12:11:20,639] Trial 0 finished with value: 0.879791497627302 and parameters: {'criterion': 'entropy', 'max_depth': 36, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.879791497627302.
[I 2025-11-17 12:11:22,148] Trial 1 finished with value: 0.5907526247203166 and parameters: {'criterion': 'log_loss', 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 11, 'max_features': 'log2'}. Best is trial 0 with value: 0.879791497627302.
[I 2025-11-17 12:11:33,236] Trial 2 finished with value: 0.8766934670895724 and parameters: {'criterion': 'log_loss', 'max_depth': 43, 'min_samples_split': 9, 'min_samples_leaf': 15, 'max_features': None}. Best is trial 0 with value: 0.879791497627302.
[I 2025-11-17 12:11:35,940] Trial 3 finished with value: 0.8814265692999926 and parameters: {'criterion': 'log_loss', 'max_dep


Best Parameters: {'criterion': 'entropy', 'max_depth': 37, 'min_samples_split': 19, 'min_samples_leaf': 4, 'max_features': None}
Best Validation Score: 0.9058420004425758

Test Accuracy: 0.9153892050670094

Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.92      0.91     63556
           2       0.93      0.93      0.93     85078
           3       0.90      0.92      0.91     10638
           4       0.85      0.78      0.81       795
           5       0.80      0.75      0.77      2941
           6       0.85      0.83      0.84      5227
           7       0.93      0.92      0.92      6069

    accuracy                           0.92    174304
   macro avg       0.88      0.86      0.87    174304
weighted avg       0.92      0.92      0.92    174304

Confusion Matrix
 [[58295  4801     6     0    65    11   378]
 [ 5258 78792   311     2   420   224    71]
 [    2   307  9758    77    41   453     0]
 [    0     1   14

In [ ]:
import joblib

joblib.dump(best_model, "decision_tree_optuna.pkl")
joblib.dump(scaler, "scaler_dt.pkl")


['scaler_dt.pkl']

In [ ]:

#Random Forest
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score

def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 80, 200),
        "max_depth": trial.suggest_int("max_depth", 5, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "class_weight": "balanced"
    }

    model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)


    return f1_score(y_valid, preds, average="macro")


# Run Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best parameters:", study.best_params)
print("Best macro F1 (validation):", study.best_value)
best_params = study.best_params
best_params["class_weight"] = "balanced"

best_rf = RandomForestClassifier(
    **best_params,
    random_state=42,
    n_jobs=-1
)

best_rf.fit(X_train, y_train)

y_pred = best_rf.predict(X_test)

print("Test Accuracy:", accuracy_score(y_test30, y_pred))
print("\nClassification Report:\n", classification_report(y_test30, y_pred))

[I 2025-11-17 12:17:15,091] A new study created in memory with name: no-name-2c325b4d-5b54-469e-b9f1-663143623b16
[I 2025-11-17 12:17:46,971] Trial 0 finished with value: 0.5135201826471038 and parameters: {'n_estimators': 146, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 0.5135201826471038.
[I 2025-11-17 12:21:49,766] Trial 1 finished with value: 0.6720761510319496 and parameters: {'n_estimators': 116, 'max_depth': 12, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': None, 'bootstrap': False}. Best is trial 1 with value: 0.6720761510319496.
[I 2025-11-17 12:23:03,157] Trial 2 finished with value: 0.7197728297406538 and parameters: {'n_estimators': 122, 'max_depth': 12, 'min_samples_split': 19, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False}. Best is trial 2 with value: 0.7197728297406538.
[I 2025-11-17 12:24:42,549] Trial 3 finished with value: 0.753148411773024 a

Best parameters: {'n_estimators': 200, 'max_depth': 20, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_features': 'log2', 'bootstrap': False}
Best macro F1 (validation): 0.8869824670485675
Test Accuracy: 0.9293418395447035

Classification Report:
               precision    recall  f1-score   support

           1       0.94      0.93      0.93     63556
           2       0.95      0.92      0.94     85078
           3       0.91      0.93      0.92     10638
           4       0.78      0.92      0.84       795
           5       0.66      0.93      0.77      2941
           6       0.79      0.94      0.86      5227
           7       0.90      0.98      0.94      6069

    accuracy                           0.93    174304
   macro avg       0.85      0.94      0.89    174304
weighted avg       0.93      0.93      0.93    174304



In [ ]:
import joblib

joblib.dump(best_rf, "rf_optuna.pkl")


['rf_optuna.pkl']

Logistic Regression


In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def objective(trial):

    # Hyperparameters to tune
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)
    solver = trial.suggest_categorical("solver", ["lbfgs", "saga"])

    # Handle valid (solver, penalty, multi_class) combinations
    #if solver == "liblinear":
        #penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
        #multi_class = "ovr"
    if solver == "lbfgs":
        penalty = "l2"              # lbfgs supports only l2 here
        multi_class = "multinomial"
    else:  # "saga"
        penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
        multi_class = "multinomial"

    model = LogisticRegression(
        C=C,
        solver=solver,
        penalty=penalty,
        multi_class=multi_class,
        class_weight="balanced",
        max_iter=500,
        n_jobs=-1,
        random_state=42
    )

    # Train on 56% train split
    model.fit(X_train, y_train)

    # Validate on 14% validation split
    preds = model.predict(X_valid)

    return accuracy_score(y_valid, preds)


# -----------------------------
#  Run Optuna Study
# -----------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("\nBest Parameters:", study.best_params)
print("Best Validation Score:", study.best_value)

# -----------------------------
#  Training Best Model on the entire 56% train split
#
# -----------------------------
best_params = study.best_params

C_best = best_params["C"]
solver_best = best_params["solver"]
penalty_best = best_params.get("penalty", "l2")

#if solver_best == "liblinear":
    #multi_class_best = "ovr"
#else:
multi_class_best = "multinomial"

best_model = LogisticRegression(
    C=C_best,
    solver=solver_best,
    penalty=penalty_best,
    multi_class=multi_class_best,
    class_weight="balanced",
    max_iter=500,
    n_jobs=-1,
    random_state=42
)

best_model.fit(X_train, y_train)

# -----------------------------
#  Final Evaluation on 30% Test Set
#  (X_test must be the scaled version of X_test30_raw)
# -----------------------------
y_pred = best_model.predict(X_test)

print("\nTest Accuracy:", accuracy_score(y_test30, y_pred))
print("\nClassification Report:\n", classification_report(y_test30, y_pred))
print("Confusion Matrix\n", confusion_matrix(y_test30, y_pred))


[I 2025-12-08 15:05:50,340] A new study created in memory with name: no-name-a230cdba-2dc8-42fd-a98a-ed064b56cd55
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
[I 2025-12-08 15:07:56,658] Trial 0 finished with value: 0.5555186742396302 and parameters: {'C': 52.56136453273163, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.5555186742396302.
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
[I 2025-12-08 15:10:02,459] Trial 1 finished with value: 0.5556047306434561 and parameters: {'C': 0.20340166009014893, 'solver': 'l


Best Parameters: {'C': 1.824150341095121, 'solver': 'saga', 'penalty': 'l2'}
Best Validation Score: 0.567640333407096


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Test Accuracy: 0.5681797319625482

Classification Report:
               precision    recall  f1-score   support

           1       0.67      0.60      0.63     63556
           2       0.77      0.52      0.62     85078
           3       0.69      0.54      0.60     10638
           4       0.23      0.88      0.37       795
           5       0.10      0.73      0.17      2941
           6       0.34      0.57      0.43      5227
           7       0.31      0.90      0.46      6069

    accuracy                           0.57    174304
   macro avg       0.44      0.67      0.47    174304
weighted avg       0.68      0.57      0.60    174304

Confusion Matrix
 [[37986 12325    11    11  2311   262 10650]
 [18291 44069  1328    72 16987  2640  1691]
 [    0    46  5725  1572   552  2743     0]
 [    0     0    69   697     0    29     0]
 [    4   651    82     0  2140    64     0]
 [    0   124  1127   647   345  2984     0]
 [  587    24     0     0    23     0  5435]]


SVM

In [ ]:


import optuna
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample

def objective(trial):

    # -----------------------------
    #   SUBSAMPLING
    # -----------------------------
    SUBSAMPLE_SIZE = 5000

    X_sub, y_sub = resample(
        X_train, y_train,
        n_samples=SUBSAMPLE_SIZE,
        stratify=y_train,
        random_state=42 + trial.number
    )

    # -----------------------------
    #  Common hyperparameters
    # -----------------------------
    C = trial.suggest_float("C", 1e-2, 1e2, log=True)
    gamma = trial.suggest_float("gamma", 1e-4, 1e-1, log=True)


    model = SVC(
        kernel="rbf",
        C=C,
        gamma=gamma,
        class_weight="balanced",
        random_state=42
    )

    # -----------------------------
    #  Train on SUBSAMPLED data
    # -----------------------------
    model.fit(X_sub, y_sub)

    # -----------------------------
    #  Validate on FULL validation set
    # -----------------------------
    preds = model.predict(X_valid)

    return accuracy_score(y_valid, preds)


# -----------------------------
#  Run Optuna
# -----------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("\nBest Parameters:", study.best_params)
print("Best Validation Score:", study.best_value)


best_params = study.best_params

best_model = SVC(
    kernel="rbf",
    C=best_params["C"],
    gamma=best_params["gamma"],
    class_weight="balanced",
    random_state=42
)

X_final, y_final = resample(
    X_train, y_train,
    n_samples=50000,
    stratify=y_train,
    random_state=42
)

best_model.fit(X_final, y_final)

# ---- evaluate on full test set ----
y_pred = best_model.predict(X_test)

print("\nTest Accuracy:", accuracy_score(y_test30, y_pred))
print("\nClassification Report:\n", classification_report(y_test30, y_pred))
print("Confusion Matrix\n", confusion_matrix(y_test30, y_pred))


[I 2025-12-10 04:27:00,780] A new study created in memory with name: no-name-6b122329-6588-4db5-99ad-008b1b0d0032
[I 2025-12-10 04:27:28,937] Trial 0 finished with value: 0.6326129182955914 and parameters: {'C': 59.24262431296422, 'gamma': 0.004774390763237564}. Best is trial 0 with value: 0.6326129182955914.
[I 2025-12-10 04:27:53,762] Trial 1 finished with value: 0.3347962921983723 and parameters: {'C': 0.03587348388094603, 'gamma': 0.0003648882648492446}. Best is trial 0 with value: 0.6326129182955914.
[I 2025-12-10 04:28:14,857] Trial 2 finished with value: 0.5714882840353077 and parameters: {'C': 8.085382184394007, 'gamma': 0.0007469778115061107}. Best is trial 0 with value: 0.6326129182955914.
[I 2025-12-10 04:28:38,662] Trial 3 finished with value: 0.37843918271003907 and parameters: {'C': 1.7204670819626897, 'gamma': 0.00015826302101423073}. Best is trial 0 with value: 0.6326129182955914.
[I 2025-12-10 04:29:02,417] Trial 4 finished with value: 0.42377861375427206 and parameter


Best Parameters: {'C': 23.78983610452667, 'gamma': 0.026393209796686398}
Best Validation Score: 0.69004942096334

Test Accuracy: 0.7274818707545437

Classification Report:
               precision    recall  f1-score   support

           1       0.75      0.76      0.75     63556
           2       0.85      0.68      0.75     85078
           3       0.76      0.74      0.75     10638
           4       0.54      0.86      0.67       795
           5       0.23      0.88      0.37      2941
           6       0.46      0.81      0.59      5227
           7       0.56      0.96      0.71      6069

    accuracy                           0.73    174304
   macro avg       0.59      0.81      0.65    174304
weighted avg       0.77      0.73      0.74    174304

Confusion Matrix
 [[48040  9933    45     0  1383   119  4036]
 [15720 57605  1589    11  7053  2528   572]
 [    0    81  7846   440   146  2125     0]
 [    0     0    65   685     0    45     0]
 [   35   191    74     0  2589

Neural Network

Without class weights

In [ ]:
# ============================
#  Imports
# ============================
import numpy as np
import optuna

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ============================
#  Prepare NN-specific labels
#  (1..7  ->  0..6)
# ============================
print("Original train labels:", np.unique(y_train))

y_train_nn = y_train - 1
y_valid_nn = y_valid - 1
y_test30_nn = y_test30 - 1

print("NN train labels after shift:", np.unique(y_train_nn))

# Features / classes info
input_dim = X_train.shape[1]
n_classes = len(np.unique(y_train_nn))


# ============================
#  Build model function
# ============================
def create_model(trial):
    model = Sequential()

    # Hyperparameters to tune
    n_layers = trial.suggest_int("n_layers", 1, 3)
    units = trial.suggest_int("units", 64, 256, step=64)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    # Input + first hidden layer
    model.add(Dense(units, activation="relu", input_shape=(input_dim,)))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    # Additional hidden layers
    for _ in range(n_layers - 1):
        model.add(Dense(units, activation="relu"))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(n_classes, activation="softmax"))

    # Compile with sparse categorical crossentropy
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


# ============================
#  Optuna objective function
#
# ============================
def objective(trial):
    # Clear previous graph between trials
    K.clear_session()

    model = create_model(trial)

    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    epochs = trial.suggest_int("epochs", 10, 40)

    early_stopping = EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True,
        verbose=0
    )

    history = model.fit(
        X_train, y_train_nn,
        validation_data=(X_valid, y_valid_nn),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0
    )

    # Best validation accuracy during training
    val_acc = max(history.history["val_accuracy"])
    return val_acc


# ============================
#  Run Optuna Study
# ============================
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)   # change 10 to 5/20 if you like

print("\nBest Parameters:", study.best_params)
print("Best Validation Accuracy:", study.best_value)


# ============================
#  Build best model
#
# ============================
best_params = study.best_params
K.clear_session()

def build_best_model(best_params):
    model = Sequential()

    n_layers = best_params["n_layers"]
    units = best_params["units"]
    dropout_rate = best_params["dropout_rate"]
    learning_rate = best_params["learning_rate"]

    model.add(Dense(units, activation="relu", input_shape=(input_dim,)))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    for _ in range(n_layers - 1):
        model.add(Dense(units, activation="relu"))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(n_classes, activation="softmax"))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

best_model = build_best_model(best_params)

early_stopping_final = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
    verbose=1
)


history_final = best_model.fit(
    X_train, y_train_nn,
    validation_split=0.1,
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    callbacks=[early_stopping_final],
    verbose=1
)


# ============================
#  Final Evaluation on Test Set
# ============================
y_pred_proba = best_model.predict(X_test)
y_pred_nn = np.argmax(y_pred_proba, axis=1)      # predicted labels in 0..6

print("\nTest Accuracy (NN, sparse):", accuracy_score(y_test30_nn, y_pred_nn))
print("\nClassification Report (NN):\n",
      classification_report(y_test30_nn, y_pred_nn))
print("Confusion Matrix (NN):\n",
      confusion_matrix(y_test30_nn, y_pred_nn))


# predictions back in original 1..7 label space:
y_pred_original = y_pred_nn + 1
print("\nFirst 20 predicted labels in original space (1..7):", y_pred_original[:20])


[I 2025-12-10 04:52:28,483] A new study created in memory with name: no-name-11955265-f5ee-4d0c-be16-13d99daf79a9


Original train labels: [1 2 3 4 5 6 7]
NN train labels after shift: [0 1 2 3 4 5 6]


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-12-10 04:55:03,779] Trial 0 finished with value: 0.8447050452232361 and parameters: {'n_layers': 3, 'units': 256, 'dropout_rate': 0.3701593716995501, 'learning_rate': 0.0011497301252093093, 'batch_size': 128, 'epochs': 38}. Best is trial 0 with value: 0.8447050452232361.
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-12-10 04:56:40,707] Trial 1 finished with value: 0


Best Parameters: {'n_layers': 3, 'units': 256, 'dropout_rate': 0.03516237373626119, 'learning_rate': 0.005989623615841982, 'batch_size': 256, 'epochs': 13}
Best Validation Accuracy: 0.8609205484390259
Epoch 1/13
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6894 - loss: 0.7705 - val_accuracy: 0.7523 - val_loss: 0.5665
Epoch 2/13
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7633 - loss: 0.5482 - val_accuracy: 0.7841 - val_loss: 0.4981
Epoch 3/13
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7876 - loss: 0.4971 - val_accuracy: 0.8039 - val_loss: 0.4592
Epoch 4/13
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8009 - loss: 0.4656 - val_accuracy: 0.8204 - val_loss: 0.4223
Epoch 5/13
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8135 - loss: 0.4401 - val_accuracy: 0.8307 - val_loss: 0.4031
Epoch 6/13
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8198 - loss: 0.4277 - val_accuracy: 0.8323 - val_loss: 0.3970
Epoch 7/13
1144/11

With Class weights to handle imbalance


In [ ]:
# ============================
#  Imports
# ============================
import numpy as np
import optuna
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ============================
#  Prepare NN-specific labels
#  (1..7  ->  0..6)
# ============================
print("Original train labels:", np.unique(y_train))

y_train_nn = y_train - 1
y_valid_nn = y_valid - 1
y_test30_nn = y_test30 - 1

print("NN train labels after shift:", np.unique(y_train_nn))

# Features / classes info
input_dim = X_train.shape[1]
n_classes = len(np.unique(y_train_nn))
classes = np.unique(y_train_nn)
class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train_nn
)
class_weight_dict = {cls: w for cls, w in zip(classes, class_weights_array)}
print(class_weight_dict)


# ============================
#  Build model function
# ============================
def create_model(trial):
    model = Sequential()

    # Hyperparameters to tune
    n_layers = trial.suggest_int("n_layers", 1, 3)
    units = trial.suggest_int("units", 64, 256, step=64)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    # Input + first hidden layer
    model.add(Dense(units, activation="relu", input_shape=(input_dim,)))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    # Additional hidden layers
    for _ in range(n_layers - 1):
        model.add(Dense(units, activation="relu"))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(n_classes, activation="softmax"))

    # Compile with sparse categorical crossentropy
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


# ============================
#  Optuna objective function
#  (Train on X_train, validate on X_valid)
# ============================
def objective(trial):
    # Clear previous graph between trials
    K.clear_session()

    model = create_model(trial)

    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    epochs = trial.suggest_int("epochs", 10, 40)

    early_stopping = EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True,
        verbose=0
    )

    history = model.fit(
        X_train, y_train_nn,
        validation_data=(X_valid, y_valid_nn),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0,
        class_weight=class_weight_dict
    )

    # Best validation accuracy during training
    val_acc = max(history.history["val_accuracy"])
    return val_acc


# ============================
#  Run Optuna Study
# ============================
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)   # change 10 to 5/20 if you like

print("\nBest Parameters:", study.best_params)
print("Best Validation Accuracy:", study.best_value)


# ============================
#  Build best model (same arch as best trial)
#  Train ONLY on X_train (no train+valid mixing)
# ============================
best_params = study.best_params
K.clear_session()

def build_best_model(best_params):
    model = Sequential()

    n_layers = best_params["n_layers"]
    units = best_params["units"]
    dropout_rate = best_params["dropout_rate"]
    learning_rate = best_params["learning_rate"]

    model.add(Dense(units, activation="relu", input_shape=(input_dim,)))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    for _ in range(n_layers - 1):
        model.add(Dense(units, activation="relu"))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(n_classes, activation="softmax"))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

best_model = build_best_model(best_params)

early_stopping_final = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train final model ONLY on X_train (as you requested)
history_final = best_model.fit(
    X_train, y_train_nn,
    validation_split=0.1,                    # small internal split from X_train itself
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    callbacks=[early_stopping_final],
    verbose=1,
    class_weight=class_weight_dict
)


# ============================
#  Final Evaluation on Test Set
# ============================
y_pred_proba = best_model.predict(X_test)
y_pred_nn = np.argmax(y_pred_proba, axis=1)      # predicted labels in 0..6

print("\nTest Accuracy (NN, sparse):", accuracy_score(y_test30_nn, y_pred_nn))
print("\nClassification Report (NN):\n",
      classification_report(y_test30_nn, y_pred_nn))
print("Confusion Matrix (NN):\n",
      confusion_matrix(y_test30_nn, y_pred_nn))


# If you want predictions back in original 1..7 label space:
y_pred_original = y_pred_nn + 1
print("\nFirst 20 predicted labels in original space (1..7):", y_pred_original[:20])


[I 2025-12-10 05:29:43,057] A new study created in memory with name: no-name-b6822447-2c45-43ee-8f88-af72e7f42842


Original train labels: [1 2 3 4 5 6 7]
NN train labels after shift: [0 1 2 3 4 5 6]
{np.int64(0): np.float64(0.39243891171855305), np.int64(1): np.float64(0.2930974376066913), np.int64(2): np.float64(2.3096078083407274), np.int64(3): np.float64(30.045802936559237), np.int64(4): np.float64(8.7222475404123), np.int64(5): np.float64(4.802237539297153), np.int64(6): np.float64(3.986351384464592)}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-12-10 05:31:17,078] Trial 0 finished with value: 0.6333628296852112 and parameters: {'n_layers': 3, 'units': 64, 'dropout_rate': 0.18001858272537063, 'learning_rate': 0.002460611706988979, 'batch_size': 128, 'epochs': 28}. Best is trial 0 with value: 0.6333628296852112.
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-12-10 05:36:05,162] Trial 1 finished with value: 0.


Best Parameters: {'n_layers': 2, 'units': 192, 'dropout_rate': 0.10025835650610726, 'learning_rate': 0.0014278451030678193, 'batch_size': 256, 'epochs': 14}
Best Validation Accuracy: 0.7453222274780273
Epoch 1/14
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.4081 - loss: 1.0751 - val_accuracy: 0.6045 - val_loss: 0.8835
Epoch 2/14
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5856 - loss: 0.6852 - val_accuracy: 0.6342 - val_loss: 0.8105
Epoch 3/14
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6188 - loss: 0.6113 - val_accuracy: 0.6553 - val_loss: 0.7452
Epoch 4/14
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6367 - loss: 0.5715 - val_accuracy: 0.6576 - val_loss: 0.7443
Epoch 5/14
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6555 - loss: 0.5359 - val_accuracy: 0.6834 - val_loss: 0.7105
Epoch 6/14
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6681 - loss: 0.5067 - val_accuracy: 0.6586 - val_loss: 0.7739
Epoch 7/14
1144/1